# POC for Elastic Search

  Steps 1: Import data as Pandas Dataframe and insert the records on Elastic Search (Local Host)

  Steps 2: Search option using Elastic Search

Note: 
Elastic Search is installed in Local with basic settings. Main purpose of this POC is implementing suitable searching pattern.

# Import data as Pandas Dataframe
For this POC this is taken from CSV file.

In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

inp_df = pd.read_csv('nicknames_proper_format.csv')
inp_df.head()

,id,name,nickname
0,1,Aaron,Erin
1,2,Aaron,Ron
2,3,Aaron,Ronnie
3,4,Abel,Ab
4,5,Abel,Abe


In [11]:
inp_df.columns = ['id', 'name_o', 'nickname']

# Insert the records on Elastic Search (Local Host)

In [12]:
# Initilize Elastic Search

from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(host='34.71.36.166', port=9200,timeout=60)

In [13]:
# Function for creating documents in ElasticSearch

def doc_generator(df,header_keys,document_index):
    """
    This generator creates documents for ElasticSearch. For each rows in pandas dataframe, one document is created.
    Rows are stored in "_source" and row number is stored as "index" 
    
    Inputs:
        df             => Input Dataframe containing Data
        header_keys    => Containing attribute names
        document_index => Index name for ElasticSearch
    """
    try:
        
        filterKeys = lambda docs , attributes : {key: document[key] for key in attributes }
        
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": document_index,
                    "_type": "_doc",
                    "_id" : f"{index}",
                    "_source": filterKeys(document, header_keys),
                }
            
            
    except StopIteration:
        return
    
    except err:
        print('Error on document generation on elastic search'+err)
    

In [14]:
# Add documents from Pandas Dataframe

header_keys = ['id', 'name_o', 'nickname']
index_name = "crime_names"

helpers.bulk(es, doc_generator(inp_df,header_keys,index_name))

# Check the Count of Records in Elastic Search 
es.indices.refresh(index = index_name)
es.cat.count(index= index_name, params={"format": "json"})

[{'epoch': '1657552342', 'timestamp': '15:12:22', 'count': '1431'}]

# Basic Search Option using Elastic Search
> Search on one attribute (Product Type = 'Small Table')

> Difference Uses of Seaerch :  match , match_phrase , query_string

In [5]:
# Creting customized methods for searching

#def search_data_from_elastic():

def build_single_query(relation,attribute,val):
    # building the elasticsearch query
    query = {
            relation: {
                attribute: val
            }
        }
    return query


def display_search(es_client, document_index,relation,attribute,val):
    """
    This function search in ElasticSearch and display the recordsets
    """
    res = es.search(index=document_index, query=build_single_query(relation,attribute,val))
    result = res['hits']['hits']
    
    for searched_records in result:
        print(searched_records['_source'])

In [6]:
# using match
display_search(es,'mozila_reviews','match','product_type','small Table')

{'product_id': 'O1', 'product_type': 'Small Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O4', 'product_type': 'Table Small', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O10', 'product_type': 'small-table', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O5', 'product_type': 'Desk small', 'product_line': 'Office-use', 'rating': 3, 'review': 'Material used Carbon steel. Colour Grey.'}
{'product_id': 'O13', 'product_type': 'table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O7', 'product_type': 'WFH Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material use

In [7]:
# using match_phrase
display_search(es,'mozila_reviews','match_phrase','product_type','small Table')

{'product_id': 'O1', 'product_type': 'Small Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O10', 'product_type': 'small-table', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}


In [8]:
# using query_string 

query = {"query_string": { "query": "(school-table) OR (Desk small)",
      "default_field": "product_type"}}
         
res = es.search(index='mozila_reviews', query=query)
result = res['hits']['hits']
    
for searched_records in result:
    print(searched_records['_source'])

{'product_id': 'O8', 'product_type': 'School Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'Good for using at School'}
{'product_id': 'O17', 'product_type': 'school-table', 'product_line': 'Office-use', 'rating': 3, 'review': 'Material used Carbon steel. Colour Grey.'}
{'product_id': 'O5', 'product_type': 'Desk small', 'product_line': 'Office-use', 'rating': 3, 'review': 'Material used Carbon steel. Colour Grey.'}
{'product_id': 'O1', 'product_type': 'Small Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O4', 'product_type': 'Table Small', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O10', 'product_type': 'small-table', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O2', 'product_type': 'Des

# Combining queries

> Search on Multiple attribute (Product Type = 'Table' and Product Line is not for Office use)

In [9]:
# Find all records where Product Type = 'Table' and Product Line is not for Office use

query = {
        "bool": {
            "must_not": {
                "match": {
                    "product_line":"Office"
                }
            },
            "should": {
                "match": {
                    "product_type": "table"
                }
            }
        }
}

res = es.search(index='mozila_reviews', query=query)
result = res['hits']['hits']
    
for searched_records in result:
    print(searched_records['_score'] ,"\t", searched_records['_source'])

0.70569915 	 {'product_id': 'F2', 'product_type': 'Wooden Table', 'product_line': 'Furniture', 'rating': 2, 'review': 'Not recommanded'}
0.5884773 	 {'product_id': 'F1', 'product_type': 'All Purpose Table', 'product_line': 'Furniture', 'rating': 4, 'review': 'Useful product for home'}
0.5884773 	 {'product_id': 'F4', 'product_type': 'Dining Table Set', 'product_line': 'Furniture', 'rating': 5, 'review': 'Useful and economical'}


# relative Search using fuzzy

In [10]:
# Find approximate records where product type is given wrong

res = es.search(index='mozila_reviews', query={"fuzzy": {"product_type": { "value": "chaer", "fuzziness": 1}}})
result = res['hits']['hits']

for searched_records in result:
    print(searched_records['_source'])

{'product_id': 'O3', 'product_type': 'Wooden Chair', 'product_line': 'Office-use', 'rating': 3, 'review': 'Not suitable for daily use'}
{'product_id': 'O12', 'product_type': 'wooden-chair', 'product_line': 'Office-use', 'rating': 2, 'review': 'Material used Carbon steel, aluminium, and stainless steel. Green Colour.'}
{'product_id': 'O15', 'product_type': 'best-chair', 'product_line': 'Office-use', 'rating': 3, 'review': 'Not suitable for daily use'}
{'product_id': 'F3', 'product_type': 'Dining Chair', 'product_line': 'Furniture', 'rating': 4, 'review': 'Good quality but overpriced'}
{'product_id': 'O6', 'product_type': 'Best Chair WFH', 'product_line': 'Office-use', 'rating': 2, 'review': 'Material used Carbon steel, aluminium, and stainless steel. Green Colour.'}


In [11]:
res = es.search(index='mozila_reviews', query={"fuzzy": {"product_type": { "value": "tablue", "fuzziness": 2}}})
result = res['hits']['hits']

for searched_records in result:
    print(searched_records['_source'])

{'product_id': 'O13', 'product_type': 'table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O1', 'product_type': 'Small Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O4', 'product_type': 'Table Small', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O7', 'product_type': 'WFH Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'This is a good Product. Material used Carbon steel, aluminium, and stainless steel'}
{'product_id': 'O8', 'product_type': 'School Table', 'product_line': 'Office-use', 'rating': 5, 'review': 'Good for using at School'}
{'product_id': 'O10', 'product_type': 'small-table', 'product_line': 'Office-use', 'rating': 4, 'review': 'Material used Carbon steel